# ETL: автоматизация подготовки данных семинары

## Урок 5. Обзор возможностей Airflow, установка и настройка

1. Скачайте файл my_first_dag.py, поместите его в папку airflow/dags. Перезапустите airflow и запустите этот dag.

2. Посмотрите логи вывода.

3. Измените этот dag так, чтобы он выводил в консоль текст «Привет + ваше имя». Посмотрите в логах, что dag выводит то, что вы хотите.

4. Скачайте файл my_second_dag.py поместите его в папку airflow/dags.

5. Объясните, что происходит в рамках этого процесса.

### Задача 1 

Скачайте файл my_first_dag.py, поместите его в папку airflow/dags. Перезапустите airflow и запустите этот dag.

![Это мой первый DAG](my_first_dag.png)

### Задача 2

Посмотрите логи вывода.

![Логи вывода](Log.png)

![Details](Details.png)

### Задача 3

Измените этот `dag` так, чтобы он выводил в консоль текст «Привет + ваше имя». Посмотрите в логах, что dag выводит то, что вы хотите.

In [ ]:
# Было
# from datetime import datetime
# from airflow import DAG
# from airflow.operators.dummy_operator import DummyOperator
# from airflow.operators.python_operator import PythonOperator

# def print_hello():
#     return 'Hello world from first Airflow DAG!'

# dag = DAG('my_first_dag', description='Hello World DAG',
#           schedule_interval='0 12 * * *',
#           start_date=datetime(2017, 3, 20), catchup=False)

# hello_operator = PythonOperator(task_id='hello_task', python_callable=print_hello, dag=dag)

# hello_operator



# Стало
from datetime import datetime
from airflow import DAG
from airflow.operators.empty import EmptyOperator
from airflow.operators.python import PythonOperator


def say_hello(name):
    return f"Привет, {name}!"


dag = DAG(
    "my_first_dag",
    description="Hello World DAG",
    schedule="@daily",
    start_date=datetime(2017, 3, 20),
    catchup=False,
)

hello_operator = PythonOperator(
    task_id="hello_task",
    python_callable=say_hello,
    op_kwargs={"name": "Шариков"},
    dag=dag,
)

hello_operator


### Задача 4

Скачайте файл my_second_dag.py поместите его в папку airflow/dags.

![my_second_dag](my_second_dag.png)

### Задача 5

Объясните, что происходит в рамках этого процесса.

In [ ]:
# Импорт библиотек
from airflow import DAG
from airflow.operators.python import PythonOperator, BranchPythonOperator
from airflow.operators.bash import BashOperator
from datetime import datetime
from random import randint

# Определение функций
# Эта функция используется для определения наилучшей модели среди трех обученных моделей 
# ('A', 'B' и 'C'). Она извлекает значения точности каждой модели через XCom (Cross-Communication) 
# и сравнивает их между собой. Если максимальная точность превышает порог 8, 
# возвращается строка "accurate"; иначе – "inaccurate".
def _choosing_best_model(ti):
    accuracies = ti.xcom_pull(task_ids=[
    'training_model_A',
    'training_model_B',
    'training_model_C'
    ])
    if max(accuracies) > 8:
        return 'accurate'
    return 'inaccurate'


# Это простая функция, которая имитирует процесс обучения модели. 
# В реальности она должна была бы тренировать модель и возвращать значение точности. 
# Здесь просто генерируется случайное число от 1 до 10, чтобы симулировать результат обучения.
def _training_model(model):
    return randint(1, 10)
# Создание DAG
# Создается новый DAG под названием "my_second_dag". Этот рабочий процесс будет запускаться 
# ежедневно начиная с 1 января 2021 года. Параметр catchup=False означает, 
# что при первом запуске будут выполнены только задачи текущего дня, без выполнения пропущенных задач 
# за предыдущие дни.
with DAG("my_second_dag", start_date=datetime(2021, 1 ,1), schedule_interval='@daily', catchup=False) as dag:
    # Задания
    # Обучение моделей
    # Создаются три задания типа PythonOperator, каждое из которых вызывает функцию _training_model 
    # для одной из моделей ('A', 'B' и 'C').
    training_model_tasks = [
        PythonOperator(
        task_id=f"training_model_{model_id}",
        python_callable=_training_model,
        op_kwargs={"model": model_id}
        ) for model_id in ['A', 'B', 'C']
    ]
    # Выбор лучшей модели
    # Задание типа BranchPythonOperator выполняет функцию _choosing_best_model, 
    # которая определяет, какая из моделей является наиболее точной.
    choosing_best_model = BranchPythonOperator(
        task_id="choosing_best_model",
        python_callable=_choosing_best_model
        )
    # Логирование результатов
    # Два задания типа BashOperator логируют результаты. Одно задание выводит сообщение "accurate", 
    # другое – "inaccurate", в зависимости от того, какой путь был выбран в задании choosing_best_model.
    accurate = BashOperator(
        task_id="accurate",
        bash_command="echo 'accurate'"
        )
    inaccurate = BashOperator(
        task_id="inaccurate",
        bash_command=" echo 'inaccurate'"
        )
    
# Поток заданий
training_model_tasks >> choosing_best_model >> [accurate, inaccurate]